In [1]:
# Importing and data
import theano.tensor as T
import theano
import sys, os
sys.path.append("../GeMpy")

# Importing GeMpy modules
import GeMpy

# Reloading (only for development purposes)
import importlib
importlib.reload(GeMpy)

# Usuful packages
import numpy as np
import pandas as pn

import matplotlib.pyplot as plt
import vtk
import random


# This was to choose the gpu
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Default options of printin
np.set_printoptions(precision = 6, linewidth= 130, suppress =  True)

#%matplotlib inline
%matplotlib inline



# Setting the extent
geo_data = GeMpy.import_data([0,10,0,10,0,10], [50,50,50])


# =========================
# DATA GENERATION IN PYTHON
# =========================
# Layers coordinates
layer_1 = np.array([[0.5,4,7], [2,4,6.5], [4,4,7], [5,4,6]])#-np.array([5,5,4]))/8+0.5
layer_2 = np.array([[3,4,5], [6,4,4],[8,4,4], [7,4,3], [1,4,6]])
layers = np.asarray([layer_1,layer_2])

# Foliations coordinates
dip_pos_1 = np.array([7,4,7])#- np.array([5,5,4]))/8+0.5
dip_pos_2 = np.array([2.,4,4])

# Dips
dip_angle_1 = float(15)
dip_angle_2 = float(340)
dips_angles = np.asarray([dip_angle_1, dip_angle_2], dtype="float64")

# Azimuths
azimuths = np.asarray([90,90], dtype="float64")

# Polarity
polarity = np.asarray([1,1], dtype="float64")

# Setting foliations and interfaces values
GeMpy.set_interfaces(geo_data, pn.DataFrame(
    data = {"X" :np.append(layer_1[:, 0],layer_2[:,0]),
            "Y" :np.append(layer_1[:, 1],layer_2[:,1]),
            "Z" :np.append(layer_1[:, 2],layer_2[:,2]),
            "formation" : np.append(
               np.tile("Layer 1", len(layer_1)), 
               np.tile("Layer 2", len(layer_2))),
            "labels" : [r'${\bf{x}}_{\alpha \, 0}^1$',
               r'${\bf{x}}_{\alpha \, 1}^1$',
               r'${\bf{x}}_{\alpha \, 2}^1$',
               r'${\bf{x}}_{\alpha \, 3}^1$',
               r'${\bf{x}}_{\alpha \, 0}^2$',
               r'${\bf{x}}_{\alpha \, 1}^2$',
               r'${\bf{x}}_{\alpha \, 2}^2$',
               r'${\bf{x}}_{\alpha \, 3}^2$',
        
                        r'${\bf{x}}_{\alpha \, 4}^2$'] }))

GeMpy.set_foliations(geo_data,  pn.DataFrame(
    data = {"X" :np.append(dip_pos_1[0],dip_pos_2[0]),
            "Y" :np.append(dip_pos_1[ 1],dip_pos_2[1]),
            "Z" :np.append(dip_pos_1[ 2],dip_pos_2[2]),
            "azimuth" : azimuths,
            "dip" : dips_angles,
            "polarity" : polarity,
            "formation" : ["Layer 1", "Layer 2"],
            "labels" : [r'${\bf{x}}_{\beta \,{0}}$',
              r'${\bf{x}}_{\beta \,{1}}$'] })) 

In [3]:
geo_data.interfaces

,X,Y,Z,formation,labels,series,order_series
0,0.5,4.0,7.0,Layer 1,"${\bf{x}}_{\alpha \, 0}^1$",Default serie,1
1,2.0,4.0,6.5,Layer 1,"${\bf{x}}_{\alpha \, 1}^1$",Default serie,1
2,4.0,4.0,7.0,Layer 1,"${\bf{x}}_{\alpha \, 2}^1$",Default serie,1
3,5.0,4.0,6.0,Layer 1,"${\bf{x}}_{\alpha \, 3}^1$",Default serie,1
4,3.0,4.0,5.0,Layer 2,"${\bf{x}}_{\alpha \, 0}^2$",Default serie,1
5,6.0,4.0,4.0,Layer 2,"${\bf{x}}_{\alpha \, 1}^2$",Default serie,1
6,8.0,4.0,4.0,Layer 2,"${\bf{x}}_{\alpha \, 2}^2$",Default serie,1
7,7.0,4.0,3.0,Layer 2,"${\bf{x}}_{\alpha \, 3}^2$",Default serie,1
8,1.0,4.0,6.0,Layer 2,"${\bf{x}}_{\alpha \, 4}^2$",Default serie,1


In [4]:
geo_data.foliations

,X,Y,Z,azimuth,dip,formation,labels,polarity,series,order_series,G_x,G_y,G_z
0,7.0,4.0,7.0,90.0,15.0,Layer 1,"${\bf{x}}_{\beta \,{0}}$",1.0,Default serie,1,0.258819,1.584810e-17,0.965926
1,2.0,4.0,4.0,90.0,340.0,Layer 2,"${\bf{x}}_{\beta \,{1}}$",1.0,Default serie,1,-0.342020,-2.094269e-17,0.939693


## Arrow with Point

In [6]:
'''
There are two alternative ways to apply the transform.
 1) Use vtkTransformPolyDataFilter to create a new transformed polydata. 
    This method is useful if the transformed polydata is needed
      later in the pipeline
    To do this, set USER_MATRIX = True
 2) Apply the transform directly to the actor using vtkProp3D's SetUserMatrix. 
    No new data is produced.
    To do this, set USER_MATRIX = False
'''
USER_MATRIX = True
 
#Create an arrow.
arrowSource = vtk.vtkArrowSource()
 
# Generate a random start and end point
random.seed(8775070)
startPoint = [0 for i in range(3)]
startPoint[0] = random.uniform(-10,10)
startPoint[1] = random.uniform(-10,10)
startPoint[2] = random.uniform(-10,10)
endPoint = [0 for i in range(3)]
endPoint[0] = random.uniform(-10,10)
endPoint[1] = random.uniform(-10,10)
endPoint[2] = random.uniform(-10,10)
 
# Compute a basis
normalizedX = [0 for i in range(3)]
normalizedY = [0 for i in range(3)]
normalizedZ = [0 for i in range(3)]
 
# The X axis is a vector from start to end
math = vtk.vtkMath()
math.Subtract(endPoint, startPoint, normalizedX)
length = math.Norm(normalizedX)
math.Normalize(normalizedX)
 
# The Z axis is an arbitrary vector cross X
arbitrary = [0 for i in range(3)]
arbitrary[0] = random.uniform(-10,10)
arbitrary[1] = random.uniform(-10,10)
arbitrary[2] = random.uniform(-10,10)
math.Cross(normalizedX, arbitrary, normalizedZ)
math.Normalize(normalizedZ)
 
# The Y axis is Z cross X
math.Cross(normalizedZ, normalizedX, normalizedY)
matrix = vtk.vtkMatrix4x4()
 
# Create the direction cosine matrix
matrix.Identity()
for i in range(3):
  matrix.SetElement(i, 0, normalizedX[i])
  matrix.SetElement(i, 1, normalizedY[i])
  matrix.SetElement(i, 2, normalizedZ[i])
 
# Apply the transforms
transform = vtk.vtkTransform()
transform.Translate(startPoint)
transform.Concatenate(matrix)
transform.Scale(length, length, length)
 
# Transform the polydata
transformPD = vtk.vtkTransformPolyDataFilter()
transformPD.SetTransform(transform)
transformPD.SetInputConnection(arrowSource.GetOutputPort())
 
#Create a mapper and actor for the arrow
mapper = vtk.vtkPolyDataMapper()
actor = vtk.vtkActor()
 
if USER_MATRIX:
    mapper.SetInputConnection(arrowSource.GetOutputPort())
    actor.SetUserMatrix(transform.GetMatrix())
else:
    mapper.SetInputConnection(transformPD.GetOutputPort())
 
actor.SetMapper(mapper)




 
# Create spheres for start and end point
sphereStartSource = vtk.vtkSphereWidget()
sphereStartSource.SetCenter(startPoint)
sphereStartMapper = vtk.vtkPolyDataMapper()
#sphereStartMapper.SetInputConnection(sphereStartSource.GetOutputPort())
sphereStart = vtk.vtkActor()
sphereStart.SetMapper(sphereStartMapper)
sphereStart.GetProperty().SetColor(1.0, 1.0, .3)



sphereEndSource = vtk.vtkSphereSource()
sphereEndSource.SetCenter(endPoint)
sphereEndMapper = vtk.vtkPolyDataMapper()
sphereEndMapper.SetInputConnection(sphereEndSource.GetOutputPort())
sphereEnd = vtk.vtkActor()
sphereEnd.SetMapper(sphereEndMapper)
sphereEnd.GetProperty().SetColor(1.0, .3, .3)
 
#Create a renderer, render window, and interactor
renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
 
    
sphereStartSource.SetInteractor(renderWindowInteractor)
sphereStartSource.SetRepresentationToSurface()
sphereStartSource.On()

    
    
    
#Add the actor to the scene
renderer.AddActor(actor)
renderer.AddActor(sphereStart)
renderer.AddActor(sphereEnd)
renderer.SetBackground(.1, .2, .3) # Background color dark blue
 
 
#Render and interact
renderWindow.Render()
renderWindowInteractor.Start()

del renderWindow, renderWindowInteractor


In [2]:
# Next we create an instance of vtkConeSource and set some of its
# properties. The instance of vtkConeSource "cone" is part of a
# visualization pipeline (it is a source process object); it produces
# data (output type is vtkPolyData) which other filters may process.
cone = vtk.vtkConeSource()
cone.SetHeight(3.0)
cone.SetRadius(1.0)
cone.SetResolution(10)

# In this example we terminate the pipeline with a mapper process object.
# (Intermediate filters such as vtkShrinkPolyData could be inserted in
# between the source and the mapper.)  We create an instance of
# vtkPolyDataMapper to map the polygonal data into graphics primitives. We
# connect the output of the cone souece to the input of this mapper.
coneMapper = vtk.vtkPolyDataMapper()
coneMapper.SetInputConnection(cone.GetOutputPort())

# Create an actor to represent the cone. The actor orchestrates rendering of
# the mapper's graphics primitives. An actor also refers to properties via a
# vtkProperty instance, and includes an internal transformation matrix. We
# set this actor's mapper to be coneMapper which we created above.
coneActor = vtk.vtkActor()
coneActor.SetMapper(coneMapper)

# Create the Renderer and assign actors to it. A renderer is like a
# viewport. It is part or all of a window on the screen and it is
# responsible for drawing the actors it has.  We also set the
# background color here.
ren1 = vtk.vtkRenderer()
ren1.AddActor(coneActor)
ren1.SetBackground(0.1, 0.2, 0.4)

# Finally we create the render window which will show up on the screen
# We put our renderer into the render window using AddRenderer. We
# also set the size to be 300 pixels by 300.
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren1)
renWin.SetSize(300, 300)
 
 # The vtkRenderWindowInteractor class watches for events (e.g., keypress,
 # mouse) in the vtkRenderWindow. These events are translated into
 # event invocations that VTK understands (see VTK/Common/vtkCommand.h
 # for all events that VTK processes). Then observers of these VTK
# events can process them as appropriate.
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# By default the vtkRenderWindowInteractor instantiates an instance
# of vtkInteractorStyle. vtkInteractorStyle translates a set of events
# it observes into operations on the camera, actors, and/or properties
# in the vtkRenderWindow associated with the vtkRenderWinodwInteractor.
# Here we specify a particular interactor style.
style = vtk.vtkInteractorStyleTrackballCamera()
iren.SetInteractorStyle(style)


# Here we use a vtkBoxWidget to transform the underlying coneActor (by
# manipulating its transformation matrix). Many other types of widgets
# are available for use, see the documentation for more details.
#
# The SetInteractor method is how 3D widgets are associated with the render
# window interactor. Internally, SetInteractor sets up a bunch of callbacks
# using the Command/Observer mechanism (AddObserver()). The place factor
# controls the initial size of the widget with respect to the bounding box
# of the input to the widget.
boxWidget = vtk.vtkBoxWidget()
boxWidget.SetInteractor(iren)
boxWidget.SetPlaceFactor(1.25)

# Place the interactor initially. The input to a 3D widget is used to
# initially position and scale the widget. The EndInteractionEvent is
# observed which invokes the SelectPolygons callback.
boxWidget.SetProp3D(coneActor)
boxWidget.PlaceWidget()

# Similar to Step2/Python/Cone2.py, we define a callback for
# interaction.  As can be seen the callback takes two arguments.  The
# first being the object that generates the event and the second
# argument the event name (which is a string).
def myCallback(widget, event_string):
    t = vtk.vtkTransform()
    boxWidget.GetTransform(t)
    boxWidget.GetProp3D().SetUserTransform(t)


# Now for every interaction event that is generated by the boxWidget,
# call our callback function.
boxWidget.AddObserver("InteractionEvent", myCallback)

# Normally the user presses the "i" key to bring a 3D widget to
# life. Here we will manually enable it so it appears with the cone.
boxWidget.On()

# Start the event loop.
iren.Initialize()
iren.Start()

del renWin, iren
# There is no explicit need to free any objects at this point.

In [7]:
renderer.SetBackground?

# Moving Points

## TODO

* camera: fit camera viewpoint automatically to model size

In [41]:
geo_data.interfaces

,X,Y,Z,formation,labels,series,order_series,x,y,z
0,6.222181,3.590821,7.951579,Layer 1,"${\bf{x}}_{\alpha \, 0}^1$",Default serie,1,NaN,NaN,NaN
1,4.174486,3.925609,5.209896,Layer 1,"${\bf{x}}_{\alpha \, 1}^1$",Default serie,1,NaN,NaN,NaN
2,4.648088,3.833807,6.402083,Layer 1,"${\bf{x}}_{\alpha \, 2}^1$",Default serie,1,NaN,NaN,NaN
3,3.898618,3.829632,7.047955,Layer 1,"${\bf{x}}_{\alpha \, 3}^1$",Default serie,1,NaN,NaN,NaN
4,4.181690,3.864074,4.483915,Layer 2,"${\bf{x}}_{\alpha \, 0}^2$",Default serie,1,NaN,NaN,NaN
5,4.310618,3.922909,3.108582,Layer 2,"${\bf{x}}_{\alpha \, 1}^2$",Default serie,1,NaN,NaN,NaN
6,4.355132,3.857064,4.000000,Layer 2,"${\bf{x}}_{\alpha \, 2}^2$",Default serie,1,8.0,3.823485,4.0
7,4.270363,3.942732,1.803516,Layer 2,"${\bf{x}}_{\alpha \, 3}^2$",Default serie,1,NaN,NaN,NaN
8,4.183521,3.703215,5.282820,Layer 2,"${\bf{x}}_{\alpha \, 4}^2$",Default serie,1,NaN,NaN,NaN


In [3]:
# sphere interaction function
def sphereCallback(obj, event):
    """
    Retrieves moved sphere coordinates and modifies coordinates
    of the respective interface point in the dataframe.
    """
    #camera.SetFocalPoint(5, 5, 5);
    #camera.SetPosition(0,5,5)
    #camera.SetViewUp(0,0,1)
    #renderer.ResetCamera()
    #print('Center: {}, {}, {}'.format(*obj.GetCenter()))
    
    # get the index of the selected point
    for i,sphere in enumerate(spheres):
        if id(sphere) == id(obj):
            # then get coordinates of the moved sphere
            coord = obj.GetCenter()
            # and modify it in the dataframe
            geo_data.interface_modify(i, X=coord[0], Y=coord[1], Z=coord[2])
    
# --------------------------------------------------------
# create renderer and render window
iren_list = []

#renderer = vtk.vtkRenderer()

renwin = vtk.vtkRenderWindow()
renwin.AddRenderer(renderer)
renwin.SetSize(700, 700)
renwin.SetWindowName('Render Window')

# create interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(renwin)

# -------------------------------------------------------
geo_data.reset_indices() # reset dataframe indices before starting anything

spheres = []
indices = []

# create sphere widgets for each interface
for index, row in geo_data.interfaces.iterrows():
    spheres.append(vtk.vtkSphereWidget())
    indices.append(index)
    spheres[-1].SetCenter(row["X"],row["Y"],row["Z"])
    spheres[-1].SetInteractor(interactor)
    spheres[-1].SetRepresentationToSurface()
    spheres[-1].On()
    
    # add observer for each sphere widget
    spheres[-1].AddObserver("InteractionEvent", sphereCallback)
# ----------------------------------------------------------------
# cube axes
cubeAxesActor = vtk.vtkCubeAxesActor()
cubeAxesActor.SetBounds(0, 10, 0, 10, 0, 10)
#cubeAxesActor.SetAxisBaseForZ(0,1,0)
#cubeAxesActor.SetAxisBaseForY(0,0,1)
cubeAxesActor.SetCamera(renderer.GetActiveCamera())

cubeAxesActor.GetTitleTextProperty(0).SetColor(1.0, 0.0, 0.0)
cubeAxesActor.GetLabelTextProperty(0).SetColor(1.0, 0.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(1).SetColor(0.0, 1.0, 0.0)
cubeAxesActor.GetLabelTextProperty(1).SetColor(0.0, 1.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(2).SetColor(1.0, 1.0, 1.0)
cubeAxesActor.GetLabelTextProperty(2).SetColor(1.0, 1.0, 1.0)
 
cubeAxesActor.DrawXGridlinesOn()
cubeAxesActor.DrawYGridlinesOn()
cubeAxesActor.DrawZGridlinesOn()


# set camera to view whole model space adequately
camera = vtk.vtkCamera();
camera.SetFocalPoint(5,5,5);
camera.SetPosition(0,5,5)
camera.SetViewUp(0,0,1)

renderer.SetActiveCamera(camera);
renderer.ResetCamera()
cubeAxesActor.SetCamera(renderer.GetActiveCamera())
# ----------------------------------------------------------------

# Define viewport ranges
xmins=[0,.5,0,.5]
xmaxs=[0.5,1,0.5,1]
ymins=[0,0,.5,.5]
ymaxs=[0.5,0.5,1,1]


for i in range(4):
    renderer = vtk.vtkRenderer()
    renwin.AddRenderer(renderer)
    renderer.SetViewport(xmins[i],ymins[i],xmaxs[i],ymaxs[i])
    renderer.AddActor(cubeAxesActor)
     
renderer.AddActor(cubeAxesActor) 
# ----------------------------------------------------------------    
# initialize window
interactor.Initialize()
interactor.Start()

del renwin, interactor

NameError: name 'renderer' is not defined

In [13]:
mapper.SetInputConnection?

# lel

In [49]:
import vtk
 
source = vtk.vtkSphereSource()
source.SetCenter(0, 0, 0)
source.SetRadius(1)
source.Update()
 
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(source.GetOutputPort())
 
actor = vtk.vtkActor()
actor.SetMapper(mapper)
 
renderer = vtk.vtkRenderer()
renderer.SetBackground(1, 1, 1)
renderer.AddActor(actor)
 
renwin = vtk.vtkRenderWindow()
renwin.AddRenderer(renderer)
 
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetInteractorStyle(vtk.vtkInteractorStyleTrackballCamera())
interactor.SetRenderWindow(renwin)
 
def DummyFunc1(obj, ev):
    print("Before Event")

def DummyFunc2(obj, ev):
    print("After Event")

# Print interator gives you a list of registered observers of the current
# interactor style
#print(interactor)
 

## adding priorities allow to control the order of observer execution
## (highest value first! if equal the first added observer is called first)
interactor.RemoveObservers('LeftButtonPressEvent')
interactor.AddObserver('LeftButtonPressEvent', DummyFunc1, 1.0)
interactor.AddObserver('LeftButtonPressEvent', DummyFunc2, -1.0)
interactor.Initialize()
interactor.Start()

Before Event
After Event


In [61]:
a = spheres[0]

In [2]:
# Call back function
def sphereCallback(obj, event):
    camera.SetFocalPoint(5, 5, 5);
    camera.SetPosition(0,5,5)
    camera.SetViewUp(0,0,1)
    renderer.ResetCamera()
    print('Center: {}, {}, {}'.format(*obj.GetCenter()))

    
# A renderer and render window
renderer = vtk.vtkRenderer()
# renderer.SetBackground(10, 10, 10)
#renderer.SetBackground(0, 0, 1)
renwin = vtk.vtkRenderWindow()
renwin.AddRenderer(renderer)
renwin.SetSize(700, 700)
renwin.SetWindowName('RW: Multiple ViewPorts')
renderer.SetViewport(0.5,0,1,1)

# An interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(renwin)




# A Sphere widget
sphereWidget = vtk.vtkSphereWidget()
sphereWidget.SetCenter(5,5,1)
sphereWidget.SetInteractor(interactor)
sphereWidget.SetRepresentationToSurface()
sphereWidget.On()

sphereWidget2 = vtk.vtkSphereWidget()
sphereWidget2.SetInteractor(interactor)
sphereWidget2.SetRepresentationToSurface()
sphereWidget2.On()
 
# Connect the event to a function
sphereWidget.AddObserver("InteractionEvent", sphereCallback)
sphereWidget2.AddObserver("InteractionEvent", sphereCallback)
 
    
    


cubeAxesActor = vtk.vtkCubeAxesActor()
cubeAxesActor.SetBounds(0, 10, 0, 10, 0, 10)
#cubeAxesActor.SetAxisBaseForZ(0,1,0)
#cubeAxesActor.SetAxisBaseForY(0,0,1)
cubeAxesActor.SetCamera(renderer.GetActiveCamera())
cubeAxesActor.GetTitleTextProperty(0).SetColor(1.0, 0.0, 0.0)
cubeAxesActor.GetLabelTextProperty(0).SetColor(1.0, 0.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(1).SetColor(0.0, 1.0, 0.0)
cubeAxesActor.GetLabelTextProperty(1).SetColor(0.0, 1.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(2).SetColor(1.0, 1.0, 1.0)
cubeAxesActor.GetLabelTextProperty(2).SetColor(1.0, 1.0, 1.0)
 
cubeAxesActor.DrawXGridlinesOn()
cubeAxesActor.DrawYGridlinesOn()
cubeAxesActor.DrawZGridlinesOn()
#if vtk.VTK_MAJOR_VERSION > 5:
#    cubeAxesActor.SetGridLineLocation(vtk.VTK_GRID_LINES_FURTHEST)
 
# cubeAxesActor.XAxisMinorTickVisibilityOff()
# cubeAxesActor.YAxisMinorTickVisibilityOff()
# cubeAxesActor.ZAxisMinorTickVisibilityOff()
 
renderer.AddActor(cubeAxesActor)

camera =vtk.vtkCamera ();
camera.SetFocalPoint(5, 5, 5);
camera.SetPosition(0,5,5)
camera.SetViewUp(0,0,1)


renderer.SetActiveCamera(camera);

renderer.ResetCamera()
cubeAxesActor.SetCamera(renderer.GetActiveCamera())


# AXES
transform = vtk.vtkTransform()
transform.Translate(1.0, 0.0, 0.0)
 
# properties of the axes labels can be set as follows
# this sets the x axis label to red
# axes->GetXAxisCaptionActor2D()->GetCaptionTextProperty()->SetColor(1,0,0);
 
# the actual text of the axis label can be changed:
# axes->SetXAxisLabelText("test");
 
#renderer.AddActor(axes)

arrowSource = vtk.vtkArrowSource()
#arrowSource.SetShaftRadius(0.01)
#arrowSource.SetTipLength(.9)
 
# Create a mapper and actor
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(arrowSource.GetOutputPort())
actor = vtk.vtkActor()
actor.SetMapper(mapper)
actor.AddObserver("InteractionEvent", sphereCallback)
renderer.AddActor(actor)

# Start
interactor.Initialize()
interactor.Start()

del renwin, interactor

Center: 4.970279721590998, 4.989296931370679, 1.208545596576589
Center: 4.970279721590998, 5.296237066673763, 1.208545596576589
Center: 4.970279721590998, 5.347393755890942, 1.208545596576589
Center: 4.970279721590998, 5.603177201976847, 1.208545596576589
Center: 4.970279721590998, 5.705490580411208, 1.208545596576589
Center: 4.970279721590998, 5.807803958845572, 1.208545596576589
Center: 4.970279721590998, 6.063587404931475, 1.208545596576589
Center: 4.970279721590998, 6.2682141618001985, 1.208545596576589
Center: 4.970279721590998, 6.626310986320464, 1.208545596576589
Center: 4.970279721590998, 6.728624364754825, 1.1573889073594081
Center: 4.970279721590998, 6.984407810840729, 1.1062322181422264
Center: 4.970279721590998, 7.291347946143814, 0.9527621504906829
Center: 4.970279721590998, 7.444818013795357, 0.9527621504906829
Center: 4.970279721590998, 7.802914838315622, 0.9016054612735056
Center: 4.970279721590998, 8.058698284401526, 0.8504487720563247
Center: 4.970279721590998, 8.1610

In [87]:
axes.SetPosition(-5, -5, - 5)

In [25]:
camera =vtk.vtkCamera ();
camera.

In [37]:
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetEventInformation()

TypeError: no overloads of SetEventInformation() take 0 arguments

In [23]:
renderer = vtk.vtkRenderer()
# renderer.SetBackground(10, 10, 10)
 
renderer.SetViewport?

### Box limit?

In [1]:
import vtk
 
# Call back function to resize the cone
def boxCallback(obj, event):
    t = vtk.vtkTransform()
    obj.GetTransform(t)
    obj.GetProp3D().SetUserTransform( t )

    
    
    
    
# A renderer and render window
renderer = vtk.vtkRenderer()
renderer.SetBackground(0, 0, 1)
# renderer.AddActor(coneActor)
 
renwin = vtk.vtkRenderWindow()
renwin.AddRenderer(renderer)
 
# An interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(renwin)



cubeAxesActor = vtk.vtkCubeAxesActor()
cubeAxesActor.SetBounds(0, 10, 0, 10, 0, 10)
cubeAxesActor.SetCamera(renderer.GetActiveCamera())
cubeAxesActor.GetTitleTextProperty(0).SetColor(1.0, 0.0, 0.0)
cubeAxesActor.GetLabelTextProperty(0).SetColor(1.0, 0.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(1).SetColor(0.0, 1.0, 0.0)
cubeAxesActor.GetLabelTextProperty(1).SetColor(0.0, 1.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(2).SetColor(0.0, 0.0, 1.0)
cubeAxesActor.GetLabelTextProperty(2).SetColor(0.0, 0.0, 1.0)
 
#cubeAxesActor.DrawXGridlinesOn()
#cubeAxesActor.DrawYGridlinesOn()
#cubeAxesActor.DrawZGridlinesOn()
if vtk.VTK_MAJOR_VERSION > 5:
    cubeAxesActor.SetGridLineLocation(vtk.VTK_GRID_LINES_FURTHEST)
 
cubeAxesActor.XAxisMinorTickVisibilityOff()
cubeAxesActor.YAxisMinorTickVisibilityOff()
cubeAxesActor.ZAxisMinorTickVisibilityOff()

  
renderer.AddActor(cubeAxesActor)

# A Box widget
boxWidget = vtk.vtkBoxWidget()
boxWidget.SetInteractor(interactor)
boxWidget.SetProp3D(cubeAxesActor)
boxWidget.SetPlaceFactor( 1.25 ) # Make the box 1.25x larger than the actor
boxWidget.PlaceWidget()
boxWidget.On()
 
# Connect the event to a function
boxWidget.AddObserver("InteractionEvent", boxCallback)
 
# Start
interactor.Initialize()
interactor.Start()

del renwin, interactor

AttributeError: module 'vtk' has no attribute 'VTK_GRID_LINES_FURTHEST'

In [58]:
cubeAxesActor.GetBounds()

(-1.0, 1.0, -1.0, 1.0, -1.0, 1.0)

In [10]:
#!/usr/bin/env python
 
import vtk
 
# Create a superquadric
superquadricSource = vtk.vtkSuperquadricSource()
superquadricSource.SetPhiRoundness(3.1)
superquadricSource.SetThetaRoundness(1.0)
superquadricSource.Update()  # needed to GetBounds later
 
renderer = vtk.vtkRenderer()
 
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(superquadricSource.GetOutputPort())
 
superquadricActor = vtk.vtkActor()
superquadricActor.SetMapper(mapper)
 
cubeAxesActor = vtk.vtkCubeAxesActor()
cubeAxesActor.SetBounds(superquadricSource.GetOutput().GetBounds())
cubeAxesActor.SetCamera(renderer.GetActiveCamera())
cubeAxesActor.GetTitleTextProperty(0).SetColor(1.0, 0.0, 0.0)
cubeAxesActor.GetLabelTextProperty(0).SetColor(1.0, 0.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(1).SetColor(0.0, 1.0, 0.0)
cubeAxesActor.GetLabelTextProperty(1).SetColor(0.0, 1.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(2).SetColor(0.0, 0.0, 1.0)
cubeAxesActor.GetLabelTextProperty(2).SetColor(0.0, 0.0, 1.0)
 
cubeAxesActor.DrawXGridlinesOn()
cubeAxesActor.DrawYGridlinesOn()
cubeAxesActor.DrawZGridlinesOn()
if vtk.VTK_MAJOR_VERSION > 5:
    cubeAxesActor.SetGridLineLocation(vtk.VTK_GRID_LINES_FURTHEST)
 
cubeAxesActor.XAxisMinorTickVisibilityOff()
cubeAxesActor.YAxisMinorTickVisibilityOff()
cubeAxesActor.ZAxisMinorTickVisibilityOff()
 
renderer.AddActor(cubeAxesActor)
renderer.AddActor(superquadricActor)
renderer.GetActiveCamera().Azimuth(30)
renderer.GetActiveCamera().Elevation(30)
 
renderer.ResetCamera()
 
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
 
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
 
renderWindow.Render()
renderWindowInteractor.Start()

del renderWindow, renderWindowInteractor

AttributeError: module 'vtk' has no attribute 'VTK_GRID_LINES_FURTHEST'

In [31]:
cubeAxesActor.SetBounds?